In [5]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding,
                          TrainingArguments, Trainer)
import evaluate, numpy as np
import os, torch

# (可选) 彻底关闭 wandb 日志
args = {
    "report_to": "none"
}

# 1. 载入数据
raw = load_dataset("multi_nli")
raw = raw.filter(lambda x: x["label"] != -1)

# 2. tokenizer
model_ckpt = "roberta-base"
tok = AutoTokenizer.from_pretrained(model_ckpt)

def tok_fn(batch):
    # truncation + 不在这里 padding
    tokenized = tok(batch["premise"], batch["hypothesis"], truncation=True)
    tokenized["labels"] = batch["label"]
    return tokenized

tok_ds = raw.map(tok_fn, batched=True, remove_columns=raw["train"].column_names)

# 3. 动态 padding collator
data_collator = DataCollatorWithPadding(tokenizer=tok)

# 4. 模型
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=3)

# 5. 训练参数
training_args = TrainingArguments(
    output_dir="./rob_ft_nli",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=200,
    **args                                # 关闭 wandb
)

# 6. 指标
acc = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return acc.compute(predictions=preds, references=labels)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,          # ✅ 指定 collator
    train_dataset=tok_ds["train"].shuffle(seed=42).select(range(50000)),
    eval_dataset=tok_ds["validation_matched"],
    compute_metrics=compute_metrics,
)

trainer.train()

# 8. 保存
trainer.save_model("./rob_ft_nli_final")
tok.save_pretrained("./rob_ft_nli_final")
print("Fine-tune 完成，模型保存在 ./rob_ft_nli_final")

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
200,1.058800
400,0.729400
600,0.606400
800,0.608800
1000,0.581000
1200,0.578200
1400,0.543600
1600,0.548300
1800,0.527100
2000,0.523800


Fine-tune 完成，模型保存在 ./rob_ft_nli_final


In [13]:
from google.colab import drive, files
drive.mount('/content/drive')
!cp -r ./rob_ft_nli_final /content/drive/MyDrive/

Mounted at /content/drive


In [19]:
from transformers import AutoConfig
cfg = AutoConfig.from_pretrained(MODEL_DIR)
ID2NAME = {f"LABEL_{i}": lbl for i, lbl in cfg.id2label.items()}
print(ID2NAME)
# {'LABEL_0': 'entailment', 'LABEL_1': 'neutral', 'LABEL_2': 'contradiction'}

{'LABEL_0': 'LABEL_0', 'LABEL_1': 'LABEL_1', 'LABEL_2': 'LABEL_2'}


In [21]:
import json, pandas as pd, torch
from transformers import pipeline
from tqdm import tqdm

MODEL_DIR = "/content/drive/MyDrive/rob_ft_nli_final"
MATCHED_JSON   = "dev_matched_sampled-1.jsonl"
MISMATCH_JSON  = "dev_mismatched_sampled-1.jsonl"

# 1. 读取数据：把 gold_label 统一转成小写字符串
def load_jsonl(path):
    samples = []
    with open(path) as f:
        for line in f:
            j = json.loads(line)
            gold = j["gold_label"]
            if isinstance(gold, int):                     # 如果是 0/1/2
                gold = ["contradiction", "neutral", "entailment"][gold]
            samples.append((j["sentence1"], j["sentence2"], gold.lower()))
    return samples

# 2. 推理 pipeline
nli = pipeline(
    "text-classification",
    model=MODEL_DIR,
    tokenizer=MODEL_DIR,
    device=0 if torch.cuda.is_available() else -1,
    batch_size=32
)

# 3. 评估并保存 CSV
ID2NAME = {"LABEL_0":"entailment",
           "LABEL_1":"neutral",
           "LABEL_2":"contradiction"}
def evaluate(samples, split):
    prompts = [f"{p}\n\nQuestion: Does this imply that \"{h}\"?\nAnswer:"
               for p, h, _ in samples]

    # 把 pipeline 返回的 label 映射成真正类别名
    preds = [ID2NAME[o["label"]] for o in nli(prompts)]
    golds = [g for _, _, g in samples]

    acc  = sum(p == g for p, g in zip(preds, golds)) / len(golds)

    # 保存 CSV，用映射后的 preds
    pd.DataFrame({
        "premise"    : [p for p,_,_ in samples],
        "hypothesis" : [h for _,h,_ in samples],
        "gold_label" : golds,
        "pred_label" : preds
    }).to_csv(f"ft_roberta_{split}.csv", index=False)

    print(f"{split} Accuracy: {acc:.2%} → 已保存 ft_roberta_{split}.csv")
    return acc

matched    = load_jsonl(MATCHED_JSON)
mismatched = load_jsonl(MISMATCH_JSON)

evaluate(matched,    "matched")
evaluate(mismatched, "mismatched")

Device set to use cuda:0


matched Accuracy: 76.44% → 已保存 ft_roberta_matched.csv
mismatched Accuracy: 77.24% → 已保存 ft_roberta_mismatched.csv


0.7724